In [13]:
# !pip install googletrans==4.0.0rc1
# pip install pyldavis
# !pip install easynmt

In [14]:
import gensim
from gensim.utils import simple_preprocess
import nltk.corpus
import pandas as pd
import gensim.corpora
from langdetect import detect
from googletrans import Translator

from nltk.corpus import stopwords
import numpy
import time

translater = Translator()

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [15]:
data = pd.read_csv("combined_profile.csv")
occupation = data['occupation']
occupation

0                Retired
1             ESTUDIANTE
2           Construction
3                Retired
4          Social worker
              ...       
8531             soldier
8532             student
8533     self employment
8534               nurse
8535             teacher
Name: occupation, Length: 8536, dtype: object

In [ ]:
value = []
for index, row in data.iterrows():
    try: 
        value.append(detect(row['occupation']))
        

    except Exception as e:
        print("Exception when handling")
        print(e)

data['occ_lan'] = value


Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception when handling
No features in text.
Exception 

In [ ]:
data['occ_lan']

In [23]:
data

,username,age,occupation,status,gender,filtered sentence,y
0,123canwe,66.0,Retired,single,male,I full fire friskier hell,0
1,123WILFREDO,28.0,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y...,0
2,1907,48.0,Construction,single,male,laid back earth good sense humor,0
3,52Jim52,70.0,Retired,divorced,male,Retired owner aerospace consuloting firm,0
4,Aaron90,28.0,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im reall...,0
...,...,...,...,...,...,...,...
8531,zak123hicks,36.0,soldier,widowed,male,Im earth caring honest going loves fun vacatio...,1
8532,zarifa2,24.0,student,single,female,I never know I write part lameSo Im ordinary n...,1
8533,zerenab,31.0,self employment,single,female,I serious mature woman good valueslearned home...,1
8534,zikki100,36.0,nurse,single,female,Im good woman looking good man One best friend...,1


In [16]:
# # from easynmt import EasyNMT

# # model = EasyNMT('opus-mt')
# # print(model.translate('ESTUDIANTE', target_lang='en'))
# from easynmt import EasyNMT

In [50]:
# def trans_english(x):
#     if detect(x) != 'en':
#         time.sleep(2)
#         translated = translater.translate(x, dest='en')
#         translator.raise_Exception = True
#         return translated.text
#     return x

In [10]:
# data['trans_occupation'] = data['occupation'].apply(lambda x:trans_english(x))

In [20]:
data

,username,age,occupation,status,gender,filtered sentence,y
0,123canwe,66.0,Retired,single,male,I full fire friskier hell,0
1,123WILFREDO,28.0,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y...,0
2,1907,48.0,Construction,single,male,laid back earth good sense humor,0
3,52Jim52,70.0,Retired,divorced,male,Retired owner aerospace consuloting firm,0
4,Aaron90,28.0,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im reall...,0
...,...,...,...,...,...,...,...
8531,zak123hicks,36.0,soldier,widowed,male,Im earth caring honest going loves fun vacatio...,1
8532,zarifa2,24.0,student,single,female,I never know I write part lameSo Im ordinary n...,1
8533,zerenab,31.0,self employment,single,female,I serious mature woman good valueslearned home...,1
8534,zikki100,36.0,nurse,single,female,Im good woman looking good man One best friend...,1


In [ ]:
data

In [40]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [79]:
odata_words = list(sent_to_words(occupation))


[['retired']]


In [76]:
def flatten(lis):
    flatList = []
    # Iterate with outer list
    for element in lis:
        if type(element) is list:
            # Check if type is list than iterate through the sublist
            for item in element:
                flatList.append(item)
        else:
            flatList.append(element)
    return flatList

In [82]:
odata_flatten = flatten(odata_words)
odata_flatten
# odata_words
print(odata_flatten[:30])

['retired', 'estudiante', 'construction', 'retired', 'social', 'worker', 'retired', 'executive', 'journalist', 'designer', 'retired', 'chef', 'entrepreneur', 'student', 'retired', 'medical', 'service', 'author', 'engineer', 'retired', 'lawyer', 'teacher', 'care', 'worker', 'cobranza', 'chofer', 'lawyer', 'disabled', 'it', 'technician']


In [90]:
import gensim.corpora as corpora
# # Create Dictionary
id2word = corpora.Dictionary(odata_words)
# Create Corpus
texts = odata_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[(0, 1)]


In [91]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.108*"business" + 0.078*"engineer" + 0.061*"nurse" + 0.054*"retired" + '
  '0.033*"abogado" + 0.028*"owner" + 0.024*"student" + 0.023*"manager" + '
  '0.018*"social" + 0.017*"worker"'),
 (1,
  '0.195*"self" + 0.191*"employed" + 0.055*"student" + 0.045*"military" + '
  '0.038*"estudiante" + 0.019*"abogado" + 0.018*"driver" + 0.018*"ventas" + '
  '0.013*"docente" + 0.012*"truck"'),
 (2,
  '0.094*"ingeniero" + 0.066*"independiente" + 0.050*"tecnico" + '
  '0.048*"engineering" + 0.026*"mecanico" + 0.026*"contador" + 0.024*"de" + '
  '0.022*"engineer" + 0.021*"nurse" + 0.017*"comerciante"'),
 (3,
  '0.072*"student" + 0.068*"construction" + 0.048*"business" + '
  '0.035*"estudiante" + 0.033*"doctor" + 0.033*"profesor" + 0.032*"military" + '
  '0.030*"enfermera" + 0.028*"chofer" + 0.026*"administrador"'),
 (4,
  '0.105*"retired" + 0.070*"student" + 0.061*"ingeniero" + 0.058*"estudiante" '
  '+ 0.049*"docente" + 0.043*"comerciante" + 0.024*"nurse" + '
  '0.023*"independiente" + 0.021*

In [127]:
data['occupation'].groupby(occupation).count()

occupation
 Accountant         5
 Administration     5
 Administrative     6
 Architect         11
 Architecture       3
                   ..
vendedor            5
ventas              3
welder              3
Área de salud       2
Преподаватель       2
Name: occupation, Length: 1016, dtype: int64